# Starting Spark Session

In [4]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("spark://3acf1e72ade9")
         .appName("load-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-42.4.0.jar")
         .getOrCreate())
sc = spark.sparkContext

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Invalid master URL: spark://3acf1e72ade9
	at org.apache.spark.util.Utils$.extractHostPortFromSparkUrl(Utils.scala:2546)
	at org.apache.spark.rpc.RpcAddress$.fromSparkURL(RpcAddress.scala:47)
	at org.apache.spark.deploy.client.StandaloneAppClient.$anonfun$masterRpcAddresses$1(StandaloneAppClient.scala:53)
	at scala.collection.ArrayOps$.map$extension(ArrayOps.scala:929)
	at org.apache.spark.deploy.client.StandaloneAppClient.<init>(StandaloneAppClient.scala:53)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.start(StandaloneSchedulerBackend.scala:123)
	at org.apache.spark.scheduler.TaskSchedulerImpl.start(TaskSchedulerImpl.scala:222)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:585)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.n

# Read Data From Mounted Volume

In [ ]:
df_movies_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/home/jovyan/work/data/movies.csv")
)

In [ ]:
df_ratings_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/home/jovyan/work/data/ratings.csv")
    .withColumnRenamed("timestamp","timestamp_epoch")
)

### Convert Timestamp column to Epoch

In [ ]:
# Convert epoch to timestamp and rating to DoubleType
from pyspark.sql.functions import from_unixtime, col, to_timestamp
from pyspark.sql.types import DoubleType

df_ratings_csv_fmt = (
    df_ratings_csv
    .withColumn('rating', col("rating").cast(DoubleType()))
    .withColumn('timestamp', to_timestamp(from_unixtime(col("timestamp_epoch"))))
)

# Load to Postgres

In [ ]:
(df_movies_csv.write
 .format("jdbc")
 .option("url", "jdbc:postgresql://mypostgres:5432/test")
 .option("dbtable", "public.movies")
 .option("user", "airflow")
 .option("password", "airflow")
 .mode("overwrite")
 .save())

In [ ]:
(df_ratings_csv_fmt
 .select([c for c in df_ratings_csv_fmt.columns if c != "timestamp_epoch"])
 .write
 .format("jdbc")
 .option("url", "jdbc:postgresql://mypostgres:5432/test")
 .option("dbtable", "public.ratings")
 .option("user", "airflow")
 .option("password", "airflow")
 .mode("overwrite")
 .save())